In [ ]:

import json
import pandas as pd
import hashlib
import random
import string
import requests
import base64
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


class HarParse(object):
    def __init__(self,hfile):
        self.filename=hfile
    
    def _customheadercol(self,parselist, key):
        value=''
        for i in parselist:
            if i['name']==key:
                value=i['value']
        return value
# get server mime type, size to cmaplre with contentlength, compression from content json
    def parse_to_df(self, requestbody=False, responsebody=False,parse_comments=True):
        self.want_req_body=requestbody
        self.want_reply_body=responsebody
        self.json_file=''
        self.dict_file={}
        with open(self.filename) as nd:
            self.json_file=nd.read()
        self.dict_file2=json.loads(self.json_file)
        self.df=pd.DataFrame()
        pd.set_option('max_colwidth', -1)
        self.df['url']=[ind['request']['url'] for ind in self.dict_file2["log"]["entries"]]
        self.df['StartedDateTime']=[ind['startedDateTime'] for ind in self.dict_file2["log"]["entries"]]
        self.df['Method']=[ind['request']['method'] for ind in self.dict_file2["log"]["entries"]]
        self.df['RequestHeaders']=[(ind['request']['headers']) for ind in self.dict_file2["log"]["entries"]]
        #df['QueryString']=[ind['request']['queryString'][0] for ind in dict_file2["log"]["entries"]]
        self.df['RequestHeaderSize']=[ind['request']['headersSize'] for ind in self.dict_file2["log"]["entries"]]
        #df['Cookies']=[ind['request']['cookies'][0] for ind in dict_file2["log"]["entries"]]
        self.df['RequestBodySize']=[ind['request']['bodySize'] for ind in self.dict_file2["log"]["entries"]]
        self.df['httpver']=[ind['request']['httpVersion'] for ind in self.dict_file2["log"]["entries"]]
        self.df['httpver_response']=[ind['response']['httpVersion'] for ind in self.dict_file2["log"]["entries"]]
        self.df['ServerIPAddress']=[ind['serverIPAddress'] for ind in self.dict_file2["log"]["entries"]]
        self.df['ResponseCode']=[ind['response']['status'] for ind in self.dict_file2["log"]["entries"]]
        self.df['ResponseHeaders']=[(ind['response']['headers']) for ind in self.dict_file2["log"]["entries"]]
        self.df['ResponseBodySize']=[ind['response']['bodySize'] for ind in self.dict_file2["log"]["entries"]]
        self.df['ResponseHeadersSize']=[ind['response']['headersSize'] for ind in self.dict_file2["log"]["entries"]]
        self.df['RedirectURL']=[ind['response']['redirectURL'] for ind in self.dict_file2["log"]["entries"]]
        #df['ResponseContentHash']=[hashlib.sha256(ind['response']['content']['text'].encode()).hexdigest() for ind in dict_file2["log"]["entries"]]
        #df['ResponseContentHash']=[contenthash(ind['response']['content']) for ind in dict_file2["log"]["entries"]]
        self.df['_cAuthorization']=[self._customheadercol(ind['request']['headers'],'Authorization') for ind in self.dict_file2["log"]["entries"]]
        if self.want_reply_body:
            self.df['ResponseContent']=[(ind['response']['content']) for ind in self.dict_file2["log"]["entries"]]
            self.df['ResponseContentType']=[ind["mimeType"] for ind in self.df['ResponseContent'].tolist()]
            self.df['ResponseContentSize']=[ind["size"] for ind in self.df['ResponseContent'].tolist()]
            self.df['ResponseContentText']=[ind["text"] for ind in self.df['ResponseContent'].tolist()] # add if statement to check for size and only then add text else ''
            self.df['ResponseContentCompression']=[ind["compression"] for ind in self.df['ResponseContent'].tolist()]
            
#            del df['ResponseContent']
                    
        if parse_comments:
            
            if self.df['httpver'].str.contains('1.0').any() or self.df['httpver_response'].str.contains('1.0').any():
                print 'HTTP 1.0 present in HAR'
            if self.df['httpver'].str.contains('2').any():
                print 'HTTP 2 present in HAR'
            if self.df['RequestHeaderSize'][self.df['RequestHeaderSize']==-1].any() or self.df['RequestBodySize'][self.df['RequestBodySize']==-1].any() or self.df['ResponseHeadersSize'][self.df['ResponseHeadersSize']==-1].any() or self.df['ResponseBodySize'][self.df['ResponseBodySize']==-1].any():
                print '-1 errors in size lengths'
            if self.want_reply_body:
                if self.df['ResponseContentCompression'][self.df['ResponseContentCompression']>0 | self.df['ResponseContentCompression'] <0 ].any():
                    print 'There is compression present'
                print 'Types of content-types present %s'%str(list(set(self.df['ResponseContentType'].tolist())))
 #add response status codes
#add request methods
            
        return self.df
    
    def _return_reqhar_header(self):
        return self.df['RequestHeaders'].tolist()
    
    def _return_reshar_header(self):
        return self.df['ResponseHeaders'].tolist()
    
    def _return_urlhar(self):
        return self.df['url'].tolist()
        
    def _make_header(self,headerls=[]):
        self.reqhead={}
        self.reqhead={x['name']:x['value'] for x in headerls}
        return self.reqhead
    
    def replay_har(self,replacedip={},bypassurl=[],overrideheader={},creds="admin:radware",ssl=True,out=['status_code','headers']):
        self.dip=replacedip
        self.bypass=bypassurl
        self.over=overrideheader
        self.creds=creds
        self.ssl=ssl
        self.out=out
        s=requests.Session()
        if self.ssl:
            prefix='https://'
        else:
            prefix='http://'
        urlls=self._return_urlhar()
        reqharls=self._return_reqhar_header()
        
        for c,i in enumerate(urlls):
            if i in self.bypass:
                pass
            else:
                tmpheaderdict=self._make_header(headerls=reqharls[c])
                for m in self.over:
                    if m in tmpheaderdict.keys():
                        tmpheaderdict[m]=self.over[m]
                if 'Authorization' in tmpheaderdict.keys() and self.creds:
                    tmpheaderdict['Authorization']='Basic %s' % base64.b64encode(self.creds)
                i=prefix+self.dip.values()[0]+i[i.find(self.dip.keys()[0])+len(self.dip.keys()[0]):]
                print i
                r = s.get(i,headers=tmpheaderdict,verify=False)
    #            print  r.content, r.status_code
                for x in self.out:
                    print getattr(r,x)
                        


        
    
    
        
        
        
